# Read Data

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


## Required Imports

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.model_selection import RepeatedStratifiedKFold, GridSearchCV
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import Embedding, LSTM, Dropout, Bidirectional, GlobalMaxPooling1D, Dense, BatchNormalization, Activation
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report

In [ ]:
path="/content/drive/MyDrive/ML_project_datasheets/Final_code/labeled_data.csv"
data=pd.read_csv(path)

In [ ]:
data.head()

,Unnamed: 0,reviewText,Review_without_stopwords,After_lemmatization,pos_tags,senti_score,Overall Sentiment
0,3878097,I thought it was depressing,thought depressing,thought depressing,"[('thought', 'NN'), ('depressing', 'VBG')]",0.125,Positive
1,1390262,One of my all time favorite flicks. An adult t...,one time favorite flicks adult themed comedy d...,one time favorite flick adult themed comedy dr...,"[('one', 'CD'), ('time', 'NN'), ('favorite', '...",1.375,Positive
2,4629692,LOVE. LOVE. LOVE. Great movie.,love love love great movie,love love love great movie,"[('love', 'VB'), ('love', 'NN'), ('love', 'NN'...",1.250,Positive
3,6433712,The product was deliver excellent and just on ...,product deliver excellent time,product deliver excellent time,"[('product', 'NN'), ('deliver', 'VB'), ('excel...",1.000,Positive
4,4837513,Terrific movie.,terrific movie,terrific movie,"[('terrific', 'JJ'), ('movie', 'NN')]",0.000,Neutral


In [ ]:
data['reviews_text_new'] = data['After_lemmatization'].copy()

In [ ]:
data.shape

(500000, 8)

In [ ]:
Data = data[['reviews_text_new', 'Overall Sentiment']]

In [ ]:
Data.head()

,reviews_text_new,Overall Sentiment
0,thought depressing,Positive
1,one time favorite flick adult themed comedy dr...,Positive
2,love love love great movie,Positive
3,product deliver excellent time,Positive
4,terrific movie,Neutral


##  Encode lables

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(Data['Overall Sentiment'])

In [ ]:
labels[0:5]

array([2, 2, 2, 2, 1])

In [ ]:
#convert colunm 'reviews_text_new' to list
corpus = Data['reviews_text_new'].tolist()

## Feature extraction

In [ ]:
# Create a CountVectorizer to convert text to a bag-of-words matrix
bow_counts = CountVectorizer()
bow_data = bow_counts.fit_transform(corpus)

In [ ]:
print(bow_data[0])

  (0, 176109)	1
  (0, 49847)	1


# Preprocess bow_data to so we can feed to LSTM

In [ ]:
# Convert the bag-of-words matrix to sequences of integers
sequences = [list(doc.nonzero()[1]) for doc in bow_data]

In [ ]:
for seq in sequences[0]:
    print(seq)

176109
49847


In [ ]:
max_sequence_length=100

In [ ]:
# Pad the sequences for equal length
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length)

In [ ]:
X = padded_sequences
y = labels

In [ ]:
X = bow_data
y = labels

# LSTM(added more layers) with random over sampling

In [ ]:
embedding_dim = 50
lstm_units = 64
output_units=3

In [ ]:
model = Sequential()
model.add(Embedding(input_dim=bow_data.shape[1], output_dim=embedding_dim, input_length=max_sequence_length))
model.add(LSTM(units=lstm_units, return_sequences=True))
model.add(Dropout(rate=0.5))
model.add(GlobalMaxPooling1D())
model.add(BatchNormalization())
model.add(Dense(units=output_units, activation='softmax'))

In [ ]:
# Print the layers
for i, layer in enumerate(model.layers):
    print(f"Layer {i + 1}: {layer.name} - {layer}")

Layer 1: embedding - <keras.src.layers.core.embedding.Embedding object at 0x7ba0f0c97cd0>
Layer 2: lstm - <keras.src.layers.rnn.lstm.LSTM object at 0x7ba0f02b7d00>
Layer 3: dropout - <keras.src.layers.regularization.dropout.Dropout object at 0x7ba0f02b6080>
Layer 4: global_max_pooling1d - <keras.src.layers.pooling.global_max_pooling1d.GlobalMaxPooling1D object at 0x7ba0f0c94160>
Layer 5: batch_normalization - <keras.src.layers.normalization.batch_normalization.BatchNormalization object at 0x7ba0f030ad40>
Layer 6: dense - <keras.src.layers.core.dense.Dense object at 0x7ba0f030bd60>


In [ ]:
from tensorflow.keras.optimizers import Adam

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model (assuming you have X_train, y_train as your training data)
model.fit(X_train_ros, y_train_ros, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
20052/20052 [==============================] - 6643s 331ms/step - loss: 0.4174 - accuracy: 0.8434 - val_loss: 0.4195 - val_accuracy: 0.8564
Epoch 2/10
20052/20052 [==============================] - 6534s 326ms/step - loss: 0.3044 - accuracy: 0.8910 - val_loss: 0.3607 - val_accuracy: 0.8731
Epoch 3/10
20052/20052 [==============================] - 6471s 323ms/step - loss: 0.2474 - accuracy: 0.9132 - val_loss: 0.3760 - val_accuracy: 0.8601
Epoch 4/10
20052/20052 [==============================] - 6244s 311ms/step - loss: 0.2151 - accuracy: 0.9255 - val_loss: 0.4176 - val_accuracy: 0.8641
Epoch 5/10
20052/20052 [==============================] - 6164s 307ms/step - loss: 0.1940 - accuracy: 0.9334 - val_loss: 0.3379 - val_accuracy: 0.8863
Epoch 6/10
20052/20052 [==============================] - 6107s 305ms/step - loss: 0.1788 - accuracy: 0.9390 - val_loss: 0.2983 - val_accuracy: 0.9009
Epoch 7/10
20052/20052 [==============================] - 6166s 307ms/step - loss: 0.1656 - ac

In [ ]:
from sklearn.metrics import classification_report

y_pred = model.predict(X_test_ros)
y_pred_classes = np.argmax(y_pred, axis=1)
# Print classification report
print(classification_report(y_test_ros, y_pred_classes))


6267/6267 [==============================] - 155s 25ms/step
              precision    recall  f1-score   support

           0       0.93      0.92      0.92     66906
           1       0.87      0.96      0.91     66948
           2       0.94      0.85      0.89     66664

    accuracy                           0.91    200518
   macro avg       0.91      0.91      0.91    200518
weighted avg       0.91      0.91      0.91    200518



# Naive Bayes

## Gaussian naive Bayes with random under sampling

In [ ]:
cv = RepeatedStratifiedKFold(n_splits=2, n_repeats=1, random_state=1)


In [ ]:
# Define the parameter grid
params_GaussianNB = {'var_smoothing': np.logspace(0, -9, num=100)}
# Create instances of Naive Bayes models
model_GaussianNB = GaussianNB()
# Create instances of GridSearchCV
gs_GaussianNB = GridSearchCV(estimator=model_GaussianNB, param_grid=params_GaussianNB, cv=cv, verbose=1, scoring='accuracy')
# Fit each model
gs_GaussianNB.fit(X_train_rus, y_train_rus)
# Get the best parameters and best model
best_params_GaussianNB = gs_GaussianNB.best_params_
best_model_GaussianNB = gs_GaussianNB.best_estimator_
# Display the results for each model
print("Best Parameters GaussianNB: ", best_params_GaussianNB)
print("Best Model GaussianNB: ", best_model_GaussianNB)

Fitting 2 folds for each of 100 candidates, totalling 200 fits
Best Parameters GaussianNB:  {'var_smoothing': 0.1}
Best Model GaussianNB:  GaussianNB(var_smoothing=0.1)


In [ ]:
# Predict on the test set

y_pred_GaussianNB = gs_GaussianNB.best_estimator_.predict(X_test_rus)

In [ ]:
print(y_pred_GaussianNB)

[1 1 1 ... 0 1 1]


In [ ]:
# Model evaluation
accuracy = accuracy_score(y_test_rus, y_pred_GaussianNB)
print(f"Accuracy: {accuracy:.2f}")

# Classification report and confusion matrix
print("\nClassification Report:")
print(classification_report(y_test_rus, y_pred_GaussianNB))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test_rus, y_pred_GaussianNB))

Accuracy: 0.45

Classification Report:
              precision    recall  f1-score   support

           0       0.50      0.36      0.42     14419
           1       0.43      0.97      0.59     14578
           2       0.45      0.00      0.01     14447

    accuracy                           0.45     43444
   macro avg       0.46      0.44      0.34     43444
weighted avg       0.46      0.45      0.34     43444


Confusion Matrix:
[[ 5221  9124    74]
 [  461 14110     7]
 [ 4666  9715    66]]


## Multinomial naive Bayes with random under sampling

In [ ]:
from sklearn.naive_bayes import MultinomialNB

# Set up cross-validation
cv = RepeatedStratifiedKFold(n_splits=2, n_repeats=1, random_state=1)

# Define the parameter grid
params_MultinomialNB = {'alpha': np.logspace(0, -9, num=100)}  # 'alpha' is used in MultinomialNB instead of 'var_smoothing'

# Create an instance of Multinomial Naive Bayes model
model_MultinomialNB = MultinomialNB()

# Create an instance of GridSearchCV
gs_MultinomialNB = GridSearchCV(estimator=model_MultinomialNB, param_grid=params_MultinomialNB, cv=cv, verbose=1, scoring='accuracy')

# Fit the model
gs_MultinomialNB.fit(X_train_rus, y_train_rus)

# Get the best parameters and best model
best_params_MultinomialNB = gs_MultinomialNB.best_params_
best_model_MultinomialNB = gs_MultinomialNB.best_estimator_

# Display the results
print("Best Parameters MultinomialNB: ", best_params_MultinomialNB)
print("Best Model MultinomialNB: ", best_model_MultinomialNB)

# Predict on the test set
y_pred_MultinomialNB = gs_MultinomialNB.best_estimator_.predict(X_test_rus)


Fitting 2 folds for each of 100 candidates, totalling 200 fits
Best Parameters MultinomialNB:  {'alpha': 1.0}
Best Model MultinomialNB:  MultinomialNB()


In [ ]:
# Model evaluation
accuracy = accuracy_score(y_test_rus, y_pred_GaussianNB)
print(f"Accuracy: {accuracy:.2f}")

# Classification report and confusion matrix
print("\nClassification Report:")
print(classification_report(y_test_rus, y_pred_GaussianNB))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test_rus, y_pred_GaussianNB))

Accuracy: 0.45

Classification Report:
              precision    recall  f1-score   support

           0       0.50      0.36      0.42     14419
           1       0.43      0.97      0.59     14578
           2       0.45      0.00      0.01     14447

    accuracy                           0.45     43444
   macro avg       0.46      0.44      0.34     43444
weighted avg       0.46      0.45      0.34     43444


Confusion Matrix:
[[ 5221  9124    74]
 [  461 14110     7]
 [ 4666  9715    66]]
